# UNet 2D

***

## _1. Imports_

In [1]:
from unet import UNet2D
from src.datasets.Nii_Gz_Dataset_3D import Dataset_NiiGz_3D
from src.utils.Experiment import Experiment
import torch
from src.losses.LossFunctions import DiceBCELoss
from src.agents.Agent_UNet import Agent 

## _2. Configure experiment_
- __AutoReload__
    - If an experiment already exists in _model\_path_ the config will __always__ be overwritten with the existing one
    - Additionally if the model has been saved previously, this state will be reloaded
- Download _hippocampus_ data from 'http://medicaldecathlon.com/' and adapt 'img_path' and 'label_path'

In [2]:
config = [{
    'img_path': r"image_path",
    'label_path': r"label_path",
    'model_path': r'Models/UNet2D_Run1',
    'device':"cuda:0",
    # Learning rate
    'lr': 1e-4,
    'lr_gamma': 0.9999,
    'betas': (0.9, 0.99),
    # Training config
    'save_interval': 100,
    'evaluate_interval': 10,
    'n_epoch': 1000,
    'batch_size': 100,
    # Data
    'input_size': (64, 64),
    'data_split': [0.7, 0, 0.3], 
}]

## _3. Choose architecture, dataset and training agent_

- _Dataset\_Nii\_Gz\_3D_ loads 3D files. If you pass a _slice_ it will be split along the according axis.

In [3]:
# Define Experiment
dataset = Dataset_NiiGz_3D(slice=2)
device = torch.device(config[0]['device'])
ca = UNet2D(in_channels=1, padding=1, out_classes=1).to(device)
agent = Agent(ca)
exp = Experiment(config, dataset, ca, agent)
exp.set_model_state('train')
dataset.set_experiment(exp)
data_loader = torch.utils.data.DataLoader(dataset, shuffle=True, batch_size=exp.get_from_config('batch_size'))
loss_function = DiceBCELoss() 

# Number of parameters
print(sum(p.numel() for p in ca.parameters() if p.requires_grad))

(64, 64)
In basic configuration threads are limited to 1 to limit CPU usage on shared Server. Add 'unlock_CPU:True' to config to disable that.
/home/jkalkhof_locale/Documents/Models/UNet2D_Run2/data_split.dt
/home/jkalkhof_locale/Documents/Models/UNet2D_Run2/models/epoch_0
36950273


## _4. Run training_

In [4]:
agent.train(data_loader, loss_function)

Epoch: 1
Dataset size: 50


/home/jkalkhof_locale/anaconda3/envs/NCA/lib/python3.8/site-packages/unet/decoding.py:146: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  half_crop = crop // 2


1 loss = 1.3361732411384581
Epoch: 2
Dataset size: 50
2 loss = 1.0679548108577728
Epoch: 3
Dataset size: 50


KeyboardInterrupt: 

## _5. Evaluate test data_

In [ ]:
agent.getAverageDiceScore()